#Databricks notebook: Bronze - Risco de Fogo(Diário)

In [0]:
# ================================================================
# Databricks notebook: Bronze - INPE Fire Risk (Diário)
# ================================================================
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from datetime import datetime
import xarray as xr
import pandas as pd
import os

spark = SparkSession.builder.getOrCreate()

In [0]:
# ================================================================
# Parâmetros recebidos via Databricks Job
# ================================================================
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema", "")
dbutils.widgets.text("table", "")
dbutils.widgets.text("path_raw", "")
dbutils.widgets.text("data_ref_carga", "")

catalog         = dbutils.widgets.get("catalog")
schema          = dbutils.widgets.get("schema")
table           = dbutils.widgets.get("table")
path_raw        = dbutils.widgets.get("path_raw")
data_ref_carga  = dbutils.widgets.get("data_ref_carga")

In [0]:
# ================================================================
# Validação de parâmetros
# ================================================================
if not data_ref_carga:
    raise ValueError("❌ Parâmetro 'data_ref_carga' não informado")

print("============================================")
print(f"📅 Data de referência: {data_ref_carga}")
print(f"📦 Catálogo destino  : {catalog}.{schema}.{table}")
print(f"📂 Caminho RAW base  : {path_raw}")
print("============================================")

In [0]:
# ================================================================
# Monta nome e caminho do arquivo esperado (.nc)
# ================================================================
data_ref_fmt = data_ref_carga.replace("-", "")
file_name = f"INPE_FireRiskModel_2.2_FireRisk_{data_ref_fmt}.nc"
file_path = f"{path_raw}/{file_name}"

print(f"🔎 Procurando arquivo: {file_path}")

In [0]:
# ================================================================
# Verifica se o arquivo existe no volume
# ================================================================
try:
    files = [f.path for f in dbutils.fs.ls(path_raw) if file_name in f.name]
except Exception as e:
    raise FileNotFoundError(f"❌ Erro ao acessar o volume RAW: {e}")

if not files:
    raise FileNotFoundError(f"⚠️ Nenhum arquivo NetCDF encontrado para {data_ref_carga}")

print(f"✅ Arquivo encontrado: {files[0]}")

In [0]:
# ================================================================
# Leitura do arquivo NetCDF (copia para /tmp e abre com xarray)
# ================================================================
local_tmp = f"/tmp/{file_name}"

try:
    print(f"📂 Copiando para leitura local: {local_tmp}")
    dbutils.fs.cp(file_path, f"file:{local_tmp}")

    if not os.path.exists(local_tmp):
        raise FileNotFoundError(f"Arquivo não copiado corretamente: {local_tmp}")

    print(f"📖 Lendo arquivo NetCDF: {local_tmp}")
    ds = xr.open_dataset(local_tmp)
    df_pandas = ds.to_dataframe().reset_index()

    if df_pandas.empty:
        raise ValueError("⚠️ Arquivo lido, mas sem dados (DataFrame vazio).")

    # Converte para DataFrame Spark
    df = spark.createDataFrame(df_pandas)
    df = df.withColumn("data_ref_carga", lit(data_ref_carga))

    print(f"✅ Linhas lidas: {df.count()}")

except Exception as e:
    raise RuntimeError(f"❌ Erro ao ler arquivo NetCDF: {e}")

In [0]:
# ================================================================
# Escrita na Tabela Bronze
# ================================================================
(
    df.write
    .format("delta")
    .mode("overwrite")
    .option("replaceWhere", f"data_ref_carga = '{data_ref_carga}'")
    .partitionBy("data_ref_carga")
    .saveAsTable(f"{catalog}.{schema}.{table}")
)

print(f"💾 Dados gravados em: {catalog}.{schema}.{table}")
print("🚀 Job Bronze finalizado com sucesso!")